In [182]:
# Import libraries

import pandas as pd, numpy as np, re

#import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
import unicodedata
import sys
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import scipy
from sklearn.model_selection import KFold

# nltk.download('stopwords')
# nltk.download('punkt')

stop_words = stopwords.words('english')

In [100]:
def clean(text):
    text=text.lower()
    text=re.sub('&lt;/?.*?&gt;',' &lt;&gt', text)
    text=re.sub('\\d|\\W+',' ',text)
    return text

# Create stop words list
stop_words = stopwords.words('english')

In [101]:
# Create Bernoulli NB classifier
bnb = BernoulliNB(class_prior=[0.25, 0.5])

# Create Multinomial NB classifier
mnb = MultinomialNB()

# Create Logistic Regression classifier (for Penalty l1 and l2)
cls1 = LogisticRegression(penalty = 'l1', solver = 'saga', multi_class = 'auto')
cls2 = LogisticRegression(penalty = 'l2', solver = 'saga', multi_class = 'auto')

In [121]:
df1 = pd.read_csv('reworked_data.csv')
df2 = pd.read_csv('reworked_data.csv')

In [111]:
df1['DESCRIPTION_NOTES'] = df1['DESCRIPTION_NOTES'].str.lower()

In [113]:
# Create a dictionary of punctuation characters
punctuation = dict.fromkeys(i for i in range(sys.maxunicode)
                            if unicodedata.category(chr(i)).startswith('P'))

# For each string, remove any punctuation characters
df1['DESCRIPTION_NOTES'] = [string.translate(punctuation) for string in df1['DESCRIPTION_NOTES']]
df1.head()

,AR_NUMBER,AR_SEVERITY,DESCRIPTION_NOTES
0,394201,2,<c> arcr reopened at request of s stewart mai...
1,397647,2,<c> cr changed from sev 3 to sev 2 and owed to...
2,396895,2,<c> mrt change request cr 396895 moved from se...
3,397010,2,<c> mrt change request cr 397010 severity move...
4,401117,2,<c> reopened at request of owed to department ...


In [183]:
def remove_stop_words(comment):
    comment = word_tokenize(comment)
    return [word for word in comment if word not in stop_words]

df1['DESCRIPTION_NOTES'] = df1['DESCRIPTION_NOTES'].apply(lambda x: remove_stop_words(x))


In [186]:
df1.tail()

,AR_NUMBER,AR_SEVERITY,DESCRIPTION_NOTES
11112,396847,8,"[<, D, >, Received, Rod, Accumulator, Trouble,..."
11113,385660,8,"[<, D, >, Some, MOC, switch, contacts, contact..."
11114,400329,8,"[<, D, >, SW-V-70B, showing, retired, status, ..."
11115,386538,8,"[<, D, >, The, EDNA, point, W140, RHR-A, Pump,..."
11116,393364,8,"[<, D, >, Valves, MS-V-19, MS-V-20, indicator,..."


In [188]:
# Apply NLTK’s PorterStemmer.
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def apply_stemmer(comment):
    # comment = word_tokenize(comment)
    return [porter.stem(word) for word in comment]

df1['DESCRIPTION_NOTES'].apply(lambda x: apply_stemmer(x))

0        [<, C, >, ar/cr, reopen, request, S., stewart,...
1        [<, C, >, CR, chang, sev, 3, sev, 2, owe, grou...
2        [<, C, >, mrt, chang, request, :, CR, 396895, ...
3        [<, C, >, mrt, chang, request, :, CR, 397010, ...
4        [<, C, >, re-open, request, owe, To, depart, ....
                               ...                        
11112    [<, D, >, receiv, rod, accumul, troubl, alarm,...
11113    [<, D, >, some, moc, switch, contact, contact,...
11114    [<, D, >, sw-v-70b, show, retir, statu, asset,...
11115    [<, D, >, the, edna, point, w140, rhr-a, pump,...
11116    [<, D, >, valv, ms-v-19, ms-v-20, indic, dial,...
Name: DESCRIPTION_NOTES, Length: 11117, dtype: object

In [189]:
from sklearn.feature_extraction.text import TfidfVectorizer
    
# Create the tf-idf feature matrix
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform([str(df1['DESCRIPTION_NOTES'])])
feature_matrix.toarray()

array([[0.10153462, 0.10153462, 0.10153462, 0.10153462, 0.10153462,
        0.10153462, 0.10153462, 0.10153462, 0.10153462, 0.10153462,
        0.10153462, 0.10153462, 0.10153462, 0.20306923, 0.10153462,
        0.10153462, 0.10153462, 0.40613847, 0.10153462, 0.10153462,
        0.10153462, 0.10153462, 0.10153462, 0.10153462, 0.10153462,
        0.20306923, 0.20306923, 0.10153462, 0.10153462, 0.10153462,
        0.20306923, 0.10153462, 0.10153462, 0.10153462, 0.10153462,
        0.10153462, 0.40613847, 0.10153462, 0.10153462, 0.10153462,
        0.20306923, 0.10153462, 0.10153462, 0.10153462, 0.10153462,
        0.10153462, 0.10153462, 0.10153462, 0.10153462, 0.10153462,
        0.10153462, 0.10153462]])

In [ ]:
#everything above this point is fine, I just got bored doing that way so I whiched up how I was doing it.

In [190]:
vectorizer = CountVectorizer()
transformed_df = vectorizer.fit_transform(df2['DESCRIPTION_NOTES'])

In [191]:
print(vectorizer.get_feature_names())

['00', '000', '0000', '0000014320', '0000015391', '0000016405', '0000016611', '0000026451', '0000026452', '0000026453', '0000026656', '0000038026', '00000721', '00002996', '0000353', '0000368917', '0000524651', '0000541368', '00005853', '00006829', '00008', '0001', '00014418', '00016487', '00016493', '00016954', '00017421', '00019475', '00019476', '00019477', '00019478', '00019479', '00019867', '0002', '00021625', '00022694', '000227', '00023592', '00023612', '00023951', '00026051', '00028655', '0003', '0004', '0005', '000509', '0006', '0006105', '0007', '0008', '0009', '000929', '000l6p08w73a', '000ng', '000psig', '001', '0010', '0011', '0012', '00120355', '00123101', '0012923', '0013', '00132063', '0015', '0016', '00167956', '0017', '00173', '00178017', '0018', '00186054', '0019', '00191', '001a', '001b', '001c', '001n6043', '001solenoid', '002', '0020', '00205327', '00211836501', '0022', '00221397', '0022203923', '0024', '00243697', '0025144994', '00252678', '0025314660', '00255805'

In [192]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [197]:
y = df2['AR_SEVERITY']
print(y)

0        2
1        2
2        2
3        2
4        2
        ..
11112    D
11113    D
11114    D
11115    D
11116    D
Name: AR_SEVERITY, Length: 11117, dtype: object


In [198]:
features_train, features_test, target_train, target_test = train_test_split(transformed_df, y, test_size=.25, random_state=6)

In [199]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB

In [200]:
#Logistic and Binary Classifier l1
classifier = LogisticRegression(penalty='l1')
model = classifier.fit(features_train, target_train)
test_predictions = model.predict(features_test)
train_predictions = model.predict(features_train)
accuracy_test = accuracy_score(target_test, test_predictions)
accuracy_train = accuracy_score(target_train , train_predictions)

In [201]:
print (accuracy_test)

0.6496402877697842


In [202]:
print(accuracy_train)

0.9574187357562672


In [176]:
#Logistic and Binary Classifier l2
classifier = LogisticRegression(penalty='l2')
model = classifier.fit(features_train, target_train)
test_predictions = model.predict(features_test)
train_predictions = model.predict(features_train)
accuracy_test = accuracy_score(target_test, test_predictions)
accuracy_train = accuracy_score(target_train , train_predictions)

In [177]:
print (accuracy_test)

0.6561151079136691


In [178]:
print(accuracy_train)

0.9943624805085762


In [179]:
#Naive Bayes Category Binary Classifier
classifier = BernoulliNB()
model = classifier.fit(features_train, target_train)
test_predictions = model.predict(features_test)
train_predictions = model.predict(features_train)
accuracy_test = accuracy_score(target_test, test_predictions)
accuracy_train = accuracy_score(target_train , train_predictions)


In [180]:
print (accuracy_test)

0.6115107913669064


In [181]:
print(accuracy_train)

0.697373155811443


In [ ]:
#The three different classifiers turned out kinda poor. They are clearly over fit. 
#They need more work! but as a one night project, these seem to be on the right track I think. 
#More data is needed for sure to have any success on this project.